In [1]:
"""
Learning Notes

I messed up a whole bunch of things in this code vs https://github.com/openai/spinningup/blob/master/spinup/examples/pytorch/pg_math/1_simple_pg.py:
1. Reward normalization
2. Didn't use "Categorical" distribution for the policy
3. Loss is something negative and huge...
4. Sampled from softmax using argmax - should have used torch.distributions.Categorical or jax.random.categorical -- which uses gumbel softmax trick to make it differentiable
5. Model was huge - i used 512x512x2 vs 32x2
6. LR was too small - 1e-4 vs 1e-2
7. Batch size was too small - 32 vs 5000 -- also samples are each time step, not each episode
8. INCORRECT OBSERVATIONS USED IN THE EXPERIENCE BUFFER - I used the newly sampled ones after running env.step
   but this was incorrect, since I needed use the previous ones...
 
"""

'\nLearning Notes\n\nI messed up a whole bunch of things in this code vs https://github.com/openai/spinningup/blob/master/spinup/examples/pytorch/pg_math/1_simple_pg.py:\n1. Reward normalization\n2. Didn\'t use "Categorical" distribution for the policy\n3. Loss is something negative and huge...\n4. Sampled from softmax using argmax - should have used torch.distributions.Categorical or jax.random.categorical -- which uses gumbel softmax trick to make it differentiable\n5. Model was huge - i used 512x512x2 vs 32x2\n6. LR was too small - 1e-4 vs 1e-2\n7. Batch size was too small - 32 vs 5000 -- also samples are each time step, not each episode\n8. INCORRECT OBSERVATIONS USED IN THE EXPERIENCE BUFFER - I used the newly sampled ones after running env.step\n   but this was incorrect, since I needed use the previous ones...\n \n'

In [2]:
import gymnasium as gym
import optax
import os
import numpy as np

from gymnasium import wrappers
from tensorflow_probability.substrates import jax as tfp
# from stable_baselines3 import PPO

In [3]:
# The environment is described in the following link:
# https://gymnasium.farama.org/environments/classic_control/cart_pole/
env = gym.make("CartPole-v1", render_mode='rgb_array')

# Record videos
trigger = lambda _: True
env = wrappers.RecordVideo(env, video_folder="./save_videos2", episode_trigger=trigger, disable_logger=True, video_length=1000)
env = wrappers.RecordEpisodeStatistics(env)

/home/anujkhare/miniconda3/envs/foo1/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/anujkhare/code/save_videos2 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [4]:
#@title Visualize the environment!
episode_over = False
obs, _ = env.reset()
print(obs)
while not episode_over:
    # action, _ = model.predict(obs, deterministic=True)
    action = env.action_space.sample()
    obs, reward, termination, truncation, info = env.step(action)
    print(action, '--->', obs, reward, termination, truncation, info)

    episode_over = termination or truncation

[-0.04455611 -0.02220803 -0.01992462 -0.0191059 ]
0 ---> [-0.04500027 -0.21703866 -0.02030674  0.26722458] 1.0 False False {}
1 ---> [-0.04934104 -0.02163287 -0.01496225 -0.03179345] 1.0 False False {}
1 ---> [-0.0497737   0.17370042 -0.01559812 -0.32915932] 1.0 False False {}
1 ---> [-0.04629969  0.3690409  -0.02218131 -0.6267201 ] 1.0 False False {}
1 ---> [-0.03891887  0.56446534 -0.03471571 -0.92630535] 1.0 False False {}
1 ---> [-0.02762957  0.76003844 -0.05324182 -1.2296927 ] 1.0 False False {}
1 ---> [-0.0124288   0.95580345 -0.07783566 -1.5385698 ] 1.0 False False {}
1 ---> [ 0.00668727  1.151771   -0.10860706 -1.8544916 ] 1.0 False False {}
0 ---> [ 0.02972269  0.9579972  -0.1456969  -1.5974113 ] 1.0 False False {}
1 ---> [ 0.04888264  1.1545147  -0.17764512 -1.9317479 ] 1.0 False False {}
0 ---> [ 0.07197293  0.96168447 -0.21628007 -1.6990079 ] 1.0 True False {'episode': {'r': 11.0, 'l': 11, 't': 0.020849}}


In [5]:
import dataclasses

@dataclasses.dataclass
class Params:
    learning_rate: float = 1e-2
    batch_size: int = 4096  # Use a large batch size for policy iteration!
    max_steps: int = 100
    obs_dim: int = 4

    # model related
    hidden_dims: list[int] = dataclasses.field(default_factory=lambda: [32])

    # initial exploration rate
    epsilon: int = 0.15
    # epsilon is decayed over time with a cosine schedule, starting at `epsilon`
    # and ending at `epsilon_min` over `max_steps` steps.
    epsilon_min: float = 0.1  # always explore at least 10% of the time

    def get_epsilon(self, step: int) -> float:
        if step > self.max_steps:
            raise ValueError("step must be less than max_steps")
        return self.epsilon_min + (self.epsilon - self.epsilon_min) * (1 + np.cos(np.pi * step / self.max_steps) ) / 2 


params = Params()

In [6]:
#@title Define the policy
import equinox as eqx
import jax

class MLP(eqx.Module):
    layers: list
    output_proj: eqx.nn.Linear

    def __init__(
        self, in_size: int, hidden_dims: list[int], out_size: int, key: jax.random.PRNGKey):
        self.layers = []
        prev_dim = in_size
        for dim in hidden_dims:
            key, layer_key = jax.random.split(key)
            self.layers.append(
                eqx.nn.Linear(
                    in_features=prev_dim,
                    out_features=dim,
                    use_bias=True,
                    key=layer_key
                )
            )
            prev_dim = dim
        self.output_proj = eqx.nn.Linear(
            in_features=prev_dim,
            out_features=out_size,
            use_bias=True,
            key=key
        )

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
            x = jax.nn.relu(x)
        # log probs (log(softmax(x))
        x = self.output_proj(x)
        return x
        # return x - jax.scipy.special.logsumexp(x)

In [7]:
@eqx.filter_jit
def policy(model: MLP, obs: np.ndarray, epsilon: float, key: jax.random.PRNGKey):
    key1, key2, key3 = jax.random.split(key, 3)

    # Find best action based on policy model
    logits = jax.vmap(model)(obs)
    # unnormalized log-probabilities are represented by the model output logits -- treat as categorical distribution
    model_actions = tfp.distributions.Categorical(logits=logits).sample(seed=key3)

    # # maybe explore based on epsilon-greedy strategy
    # p_explore = jax.random.uniform(key1, (len(obs),))
    # random_actions = jax.random.randint(key2, (len(obs),), minval=0, maxval=2)

    # final_actions = jax.numpy.where(p_explore < epsilon, random_actions, model_actions)
    final_actions = model_actions
    return final_actions, model_actions

@dataclasses.dataclass
class SampleOutput:
    observations: np.ndarray = dataclasses.field(default_factory=lambda: np.array([[]]))
    actions: np.ndarray = dataclasses.field(default_factory=lambda: np.array([]))
    rewards: np.ndarray = dataclasses.field(default_factory=lambda: np.array([]))
    rewards_to_go: np.ndarray = dataclasses.field(default_factory=lambda: np.array([]))
    valid: np.ndarray = dataclasses.field(default_factory=lambda: np.array([]))
    avg_expected_reward: float = 0.0
    avg_action: float = 0.0
    avg_episode_length: float = 0.0
    num_episodes: int = 0

    def num_samples(self):
        return len(self.observations)
    
    def __add__(self, other):
        def _weighted_sum(attr):
            return (getattr(self, attr) * self.num_samples() + getattr(other, attr) * other.num_samples()) / (self.num_samples() + other.num_samples())
        return SampleOutput(
            observations=np.concatenate([self.observations, other.observations]),
            actions=np.concatenate([self.actions, other.actions]),
            rewards=np.concatenate([self.rewards, other.rewards]),
            rewards_to_go=np.concatenate([self.rewards_to_go, other.rewards_to_go]),
            valid=np.concatenate([self.valid, other.valid]),
            num_episodes=self.num_episodes + other.num_episodes,
            avg_expected_reward=_weighted_sum('avg_expected_reward'),
            avg_action=_weighted_sum('avg_action'),
            avg_episode_length=_weighted_sum('avg_episode_length')
        )

def sample_from_policy(envs: gym.vector.VectorEnv, model: MLP, epsilon: float, key: jax.random.PRNGKey):
    """Run all envs in parallel, return the trajectory of each env."""
    obs, _ = envs.reset()
    episode_over = np.array([False] * envs.num_envs)

    # Track the actions, rewards, and gradients for policy optimization
    trajectory_observations = []
    trajectory_rewards = []
    trajectory_actions = []
    trajectory_valid = []

    # iterate through the vectorized environments
    while np.any(~episode_over):
        trajectory_observations.append(obs)

        key_used, key = jax.random.split(key)
        actions, _ = policy(model, obs, epsilon, key_used)
        obs, rewards, terminated, truncated, _ = envs.step(np.array(actions))
        episode_over = episode_over | terminated | truncated

        # Mask out everything for episodes that are over
        mask = ~episode_over
        trajectory_rewards.append(rewards)
        trajectory_actions.append(actions)
        trajectory_valid.append(mask)

    trajectory_observations = np.einsum('Tbd->bTd', np.stack(trajectory_observations))
    trajectory_actions = np.stack(trajectory_actions).T
    trajectory_rewards = np.stack(trajectory_rewards).T
    trajectory_valid = np.stack(trajectory_valid).T

    # compute the rewards to go
    rewards_to_go = np.where(trajectory_valid, trajectory_rewards, 0)[:, ::-1].cumsum(axis=-1)[:, ::-1]

    avg_expected_reward = np.mean(rewards_to_go[:, 0])  # average expected rewards for trajectories
    avg_action = np.mean(trajectory_actions)
    avg_episode_length = np.sum(trajectory_valid, axis=-1).mean()
    num_episodes = len(trajectory_observations)

    # Flatten all the (state, action, reward) tuples across episodes 
    # it doesn't matter which episode they came from (as long as we keep a track of the rewards!)
    trajectory_valid = trajectory_valid.ravel()  # (T*B, )

    return SampleOutput(
        observations=trajectory_observations.reshape(-1, trajectory_observations.shape[-1])[trajectory_valid],
        actions=trajectory_actions.ravel()[trajectory_valid],
        rewards=trajectory_rewards.ravel()[trajectory_valid],
        rewards_to_go=rewards_to_go.ravel()[trajectory_valid],
        valid=trajectory_valid,
        avg_expected_reward=avg_expected_reward,
        avg_action=avg_action,
        avg_episode_length=avg_episode_length,
        num_episodes=num_episodes
    )

In [8]:
def loss_fn(model, obs, actions, weights):
    logits = jax.vmap(model)(obs)
    log_probs = tfp.distributions.Categorical(logits=logits).log_prob(actions)

    # the loss is technically the log likelihood of the actions weighted by the rewards-to-go
    return -jax.numpy.sum(log_probs * weights) / len(obs)

@eqx.filter_jit
def update_model(model, obs, actions, weights, optimizer, opt_state):
    loss, grad = eqx.filter_value_and_grad(loss_fn)(model, obs, actions, weights)
    updates, opt_state = optimizer.update(grad, opt_state, eqx.filter(model, eqx.is_array))
    model = eqx.apply_updates(model, updates)
    return model, opt_state, loss

In [9]:
import pdb

@dataclasses.dataclass
class SampleAndUpdatePolicyOutputs:
    model: MLP
    opt_state: optax.OptState
    loss: float
    avg_expected_reward: float
    avg_action: float
    avg_episode_length: float
    num_samples: int


def sample_batch_and_update_policy(
    envs: gym.vector.VectorEnv,
    model: MLP, optimizer: optax.GradientTransformation,
    opt_state: optax.OptState,
    batch_size: int,
    epsilon: float,
    key: jax.random.PRNGKey):
    # Sample at least "batch_size" time steps (state,action,reward)s from the policy
    samples = None
    while True:
        key_used, key = jax.random.split(key)
        new_samples = sample_from_policy(envs, model, epsilon, key_used)
        if samples is None:
            samples = new_samples
        else:
            samples = samples + new_samples
        if samples.num_samples() >= batch_size:
            break
    # print(f"Sampled {samples.num_samples()} samples from {samples.num_episodes} episodes")

    # Compute the loss and the gradient
    model, opt_state, loss = update_model(
        model,
        samples.observations,
        samples.actions,
        samples.rewards_to_go,
        optimizer,
        opt_state)
    return SampleAndUpdatePolicyOutputs(
        model=model, opt_state=opt_state,
        loss=loss,
        avg_expected_reward=samples.avg_expected_reward,
        avg_action=samples.avg_action,
        avg_episode_length=samples.avg_episode_length,
        num_samples=samples.num_samples(),
    )


def train(envs, model, optimizer, params, key: jax.random.PRNGKey):
    opt_state = optimizer.init(eqx.filter(model, eqx.is_array))

    for step in range(params.max_steps):
        key_used, key = jax.random.split(key)
        outputs = sample_batch_and_update_policy(
            envs, model, optimizer, opt_state, params.batch_size, params.get_epsilon(step), key_used)
        model, opt_state = outputs.model, outputs.opt_state
        if step % 1 == 0:
            print(f"Step {step}: Loss {outputs.loss} Avg Reward: {outputs.avg_expected_reward} Avg Episode Len: {outputs.avg_episode_length}")
            print(f"Batch size: {outputs.num_samples} Avg action: {outputs.avg_action}")
            # print(f"New epsilon: {params.get_epsilon(step)}")
    return model, opt_state

In [10]:
#@title Vectorize the enviroment such that we can sample multiple trajectories concurrently
envs = gym.make_vec("CartPole-v1", num_envs=32, vectorization_mode='vector_entry_point')

In [11]:
# opt_state = optim.init(eqx.filter(model, eqx.is_array))
optim  = optax.adam(params.learning_rate)
model = MLP(
    in_size=4,
    hidden_dims=params.hidden_dims + [2],
    out_size=2,
    key=jax.random.PRNGKey(12513)
)

In [12]:
train(envs, model, optim, params, jax.random.PRNGKey(812342))

2024-12-31 10:03:21.985288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735639401.998689 1735472 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735639402.002472 1735472 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Step 0: Loss 9.580428123474121 Avg Reward: 21.104222039123965 Avg Episode Len: 21.104222039123965
Batch size: 4703 Avg action: 0.5674805225324212
Step 1: Loss 9.169675827026367 Avg Reward: 20.72004785853342 Avg Episode Len: 20.72004785853342
Batch size: 4623 Avg action: 0.5566305391264313
Step 2: Loss 10.089892387390137 Avg Reward: 22.224314151392527 Avg Episode Len: 22.224314151392527
Batch size: 4201 Avg action: 0.5353013828589164
Step 3: Loss 10.432099342346191 Avg Reward: 23.31463500563698 Avg Episode Len: 23.31463500563698
Batch size: 4435 Avg action: 0.5259911205168764
Step 4: Loss 11.014314651489258 Avg Reward: 24.926968085106385 Avg Episode Len: 24.926968085106385
Batch size: 4700 Avg action: 0.5293880168274832
Step 5: Loss 12.52457046508789 Avg Reward: 25.885930696661827 Avg Episode Len: 25.885930696661827
Batch size: 4134 Avg action: 0.5248507545547348
Step 6: Loss 13.75375747680664 Avg Reward: 29.79821220004247 Avg Episode Len: 29.79821220004247
Batch size: 4709 Avg action: 

(MLP(
   layers=[
     Linear(
       weight=f32[32,4],
       bias=f32[32],
       in_features=4,
       out_features=32,
       use_bias=True
     ),
     Linear(
       weight=f32[2,32],
       bias=f32[2],
       in_features=32,
       out_features=2,
       use_bias=True
     )
   ],
   output_proj=Linear(
     weight=f32[2,2],
     bias=f32[2],
     in_features=2,
     out_features=2,
     use_bias=True
   )
 ),
 (ScaleByAdamState(count=Array(100, dtype=int32), mu=MLP(
    layers=[
      Linear(
        weight=f32[32,4],
        bias=f32[32],
        in_features=4,
        out_features=32,
        use_bias=True
      ),
      Linear(
        weight=f32[2,32],
        bias=f32[2],
        in_features=32,
        out_features=2,
        use_bias=True
      )
    ],
    output_proj=Linear(
      weight=f32[2,2],
      bias=f32[2],
      in_features=2,
      out_features=2,
      use_bias=True
    )
  ), nu=MLP(
    layers=[
      Linear(
        weight=f32[32,4],
        bias=f32[32

In [13]:
sorted(os.listdir("./save_videos2"))

['rl-video-episode-0.mp4',
 'rl-video-episode-1.mp4',
 'rl-video-episode-10.mp4',
 'rl-video-episode-11.mp4',
 'rl-video-episode-12.mp4',
 'rl-video-episode-13.mp4',
 'rl-video-episode-14.mp4',
 'rl-video-episode-2.mp4',
 'rl-video-episode-3.mp4',
 'rl-video-episode-4.mp4',
 'rl-video-episode-5.mp4',
 'rl-video-episode-6.mp4',
 'rl-video-episode-7.mp4',
 'rl-video-episode-8.mp4',
 'rl-video-episode-9.mp4']